In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

In [ ]:
AUDIO_PATH = './AudioWAV/'  

In [ ]:
def load_data():
    file_emotion = []
    file_path = []
    directory_list = os.listdir(AUDIO_PATH)

    for file in directory_list:
        part = file.split('_')
        if part[2] == 'SAD':
            file_emotion.append('sad')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        file_path.append(AUDIO_PATH + file)

    crema_df = pd.DataFrame({'Emotions': file_emotion, 'Path': file_path})
    return crema_df

In [ ]:
def extract_features(data, sample_rate):
    result = np.array([])
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=20).T, axis=0)
    result = np.hstack((result, mfcc))
    return result

In [ ]:
def get_features(path):
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    result = np.array(extract_features(data, sample_rate))
    noise_data = data + 0.035 * np.random.uniform() * np.amax(data) * np.random.normal(size=data.shape[0])
    result = np.vstack((result, extract_features(noise_data, sample_rate)))

    return result

In [ ]:
def prepare_dataset():
    crema_df = load_data()
    X, Y = [], []
    with ThreadPoolExecutor() as executor:
        futures = []
        for path, emotion in zip(crema_df.Path, crema_df.Emotions):
            futures.append(executor.submit(get_features, path))
        
        for future, emotion in zip(futures, crema_df.Emotions):
            features = future.result()
            for feature in features:
                X.append(feature)
                Y.append(emotion)

    return np.array(X), np.array(Y)

In [ ]:
def preprocess_data():
    X, Y = prepare_dataset()
    encoder = OneHotEncoder()
    Y = encoder.fit_transform(Y.reshape(-1, 1)).toarray()
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42, shuffle=True)
    return x_train, x_test, y_train, y_test

In [ ]:
if __name__ == "__main__":
    x_train, x_test, y_train, y_test = preprocess_data()
    
    
    features_df = pd.DataFrame(x_train)
    features_df['label'] = y_train.argmax(axis=1)  
    features_df.to_csv("features.csv", index=False)
    
    print("Data preprocessing complete. Features saved to features.csv")